In [40]:
import pandas as pd
import numpy as np

In [41]:
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)

In [42]:
headers = df.columns.tolist()

# dropping general data from sex column (both sexes data) and age column (all ages data)
df = df[df['Sex'] != 'Both sexes']
df = df[df['Single Year of Age'] != 'All ages']
#print(df.head())
#print(df.tail())

# dropping unnecessary columns
columns_to_drop = ['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear', 'C02199V02655', 'C02076V03371', 'C03789V04537', 'Administrative Counties', 'UNIT']
df = df.drop(columns=columns_to_drop)

# tidying up age values
df['Single Year of Age'] = df['Single Year of Age'].replace({'100 years and over': '100'})
df['Single Year of Age'] = df['Single Year of Age'].replace({'Under 1 year': '0'})

#print(df.head())

# removing the words "year" or "years" from every row in the age column as well as any whitespace and then converting every value in column to an interger
# reference: https://www.sparkcodehub.com/pandas/data-cleaning/string-replace
df['Single Year of Age'] = df['Single Year of Age'].str.replace('years', '')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('year', '')
df['Single Year of Age'] = df['Single Year of Age'].str.strip()
df['Single Year of Age'] = df['Single Year of Age'].astype(int)
df = df.rename(columns={'VALUE': 'Population', 'Single Year of Age': 'Age'})

#df['Single Year of Age'] = df['Single Year of Age'].astype(int)

df.to_csv('irish_population_by_age_and_sex.csv', index=False)



In [43]:
# calculating the weighted mean age and difference between the sexes by age
# https://dev.to/chrisgreening/calculating-weighted-averages-with-numpy-and-python-4m79
# https://numpy.org/doc/stable/reference/generated/numpy.average.html
# https://www.geeksforgeeks.org/python/grouping-and-aggregating-with-pandas/

# Firstly, I need to group the data by sex and age, collapsing duplicate rows so that each Sex/Age pair has a single population value
df_grouped = df.groupby(["Sex", "Age"], as_index=False)["Population"].sum()

# Then, I can calculate the weighted mean age with a for loop
# df_grouped.groupby groups the dataframe into two sub-dataframes by the variables within the Sex column
# the for loop iterates through each sub-dataframe, calculating the weighted mean age for each sex
for sex, group in df_grouped.groupby("Sex"):
    weighted_mean_age = np.average(group["Age"], weights=group["Population"])
    # prints mean age rounded to 2 decimal places
    print(f"Weighted mean age for {sex}: {weighted_mean_age:.2f}")







Weighted mean age for Female: 38.94
Weighted mean age for Male: 37.74


In [44]:

# here I calculate the difference in population between the sexes by age and save it to a new CSV file
# this creates a pivot table from the grouped dataframe, with Age as the index, Sex as the columns and Population as the values
df_pivot = df_grouped.pivot(index='Age', columns='Sex', values='Population')

# this simplpy subtracts the Population value in the Male column for a given age bracket from the Population value in the Female for same
df_pivot['Difference'] = df_pivot['Male'] - df_pivot['Female']

# saving the result to a csv file
df_pivot.to_csv('irish_population_difference_by_age_and_sex.csv')

# I also print descriptive statistics for the difference column to see how the population difference varies by age
print(df_pivot['Difference'].describe())

# then to generate a single 'difference' value for the entire population, I sum the absolute values of the differences for each age bracket
total_difference = df_pivot['Difference'].abs().sum()

# I write a quick if statement so that if the difference is positive, it prints a statement saying that there are more males by that amount, and if negative, it prints a statement saying there
if total_difference > 0:
    print(f'There are {total_difference} more males than females')
elif total_difference < 0:
    print(f'There are {abs(total_difference)} more females than males')
else:
    print('The population of males to females is balanced')

count     101.000000
mean    -1188.930693
std      2838.681801
min     -7686.000000
25%     -3142.000000
50%     -1566.000000
75%      1024.000000
max      3938.000000
Name: Difference, dtype: float64
There are 264466 more males than females


In [45]:
age = 35

# below I create a function that groups the Population values within a 5 year range of a given age and then calculates the difference in population between
# the sexes foer that age range
# the function defines a max and min age, then creates a new dataframe based on filtering the grouped dataframe for ages within that range
# then it creates a pivot table from that filtered dataframe and calculates the difference in population between the sexes
# then, as above, it creates a new column in the pivot table for calculating the difference in each age within the range
# finally, it sums the absolute values of the differences for each age within the range and prints the result

# I reuse the same if statement as above to print whether there are more males or females overall

def population_difference_by_age_range(age):
    age_min = age - 5
    age_max = age + 5
    df_range = df_grouped[(df_grouped['Age'] >= age_min) & (df_grouped['Age'] <= age_max)]
    df_pivot_range = df_range.pivot(index='Age', columns='Sex', values='Population')
    df_pivot_range['Difference'] = df_pivot_range['Male'] - df_pivot_range['Female']
    total_difference_range = df_pivot_range['Difference'].abs().sum()

    if total_difference > 0:
        print(f'There are {total_difference_range} more males than females in this age range')
    elif total_difference < 0:
        print(f'There are {abs(total_difference_range)} more females than males in this age range')
    else:
        print('The population of males to females is balanced')


population_difference_by_age_range(35)



There are 60952 more males than females in this age range


In [62]:
# writing the code to work out which region in Ireland has the biggest population difference between the sexes in that age group

url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df_2 = pd.read_csv(url)

headers = df_2.columns.tolist()
#print(headers)
# print the first few rows of the dataframe
#print(df_2_2.head())

columns_to_drop = ['STATISTIC', 'Statistic Label', 'TLIST(A1)', 'CensusYear', 'C02199V02655', 'C02076V03371', 'C03789V04537', 'UNIT']
df_2 = df_2.drop(columns=columns_to_drop)

#print(df_2_2.head())

#tidying up values as I did previously
# tidying up age values
df_2 = df_2[df_2['Sex'] != 'Both sexes']
df_2 = df_2[df_2['Single Year of Age'] != 'All ages']
df_2['Single Year of Age'] = df_2['Single Year of Age'].replace({'100 years and over': '100'})
df_2['Single Year of Age'] = df_2['Single Year of Age'].replace({'Under 1 year': '0'})
df_2['Single Year of Age'] = df_2['Single Year of Age'].str.replace('years', '')
df_2['Single Year of Age'] = df_2['Single Year of Age'].str.replace('year', '')
df_2['Single Year of Age'] = df_2['Single Year of Age'].str.strip()
df_2['Single Year of Age'] = df_2['Single Year of Age'].astype(int)
df_2 = df_2.rename(columns={'VALUE': 'Population', 'Single Year of Age': 'Age', 'Administrative Counties': 'Region'})


#(df_2.head())

df_2_pivot = df_2.pivot_table(index=['Region', 'Age'], columns='Sex', values='Population')

#print(df_2_pivot.head())

# adjusting my earlier function to also return the region with the highest difference
def region_population_difference_by_age(age):
    # establishing range as before
    age_min = age - 5
    age_max = age + 5
    # whittling down exisitng dataframe to only relevant data
    df_region_age = df_2[(df_2['Age'] >= age_min) & (df_2['Age'] <= age_max)]
    df_region_pivot = df_region_age.pivot_table(index=['Region', 'Age'], columns='Sex', values='Population')
    # next I need to calculate the difference column as before
    df_region_pivot['Difference'] = df_region_pivot['Male'] - df_region_pivot['Female']
    # now I group by region and sum the values in the differnce column
    df_region_difference = df_region_pivot.groupby('Region')['Difference'].sum()
    # then I find the region with the highest absolute difference value
    max_difference_region = df_region_difference.idxmax()
    # then, depending on whether the difference is positive or negative, I print the appropriate statement
    if df_region_difference[max_difference_region] > 0:
        print(f'The region with the highest population difference between the sexes in this age range is {max_difference_region}, with {df_region_difference[max_difference_region]} more males than females')
    elif df_region_difference[max_difference_region] < 0:
        print(f'The region with the highest population difference between the sexes in this age range is {max_difference_region}, with {abs(df_region_difference[max_difference_region])} more females than males')
    else:
        print('The population of males to females is balanced in all regions at this age range')


region_population_difference_by_age(35)





The region with the highest population difference between the sexes in this age range is Dublin City Council, with 1036.0 more males than females
